In [24]:
import pandas as pd
import numpy as np

In [25]:
import plotly.graph_objects as go
from plotly.offline import plot

In [26]:
# Load data
data = pd.read_csv('C:\\Heri\\GitHub\\Crypto\\02-Coingecko\\data-crypto.csv')  # Replace with your file name
data['Date'] = pd.to_datetime(data['timestamp'])
data.set_index('Date', inplace=True)

In [27]:
# Columns: Ripple and Dogecoin prices
ripple_prices = data['ripple']
dogecoin_prices = data['dogecoin']

In [28]:
# Calculate daily returns
ripple_returns = ripple_prices.pct_change().dropna()
dogecoin_returns = dogecoin_prices.pct_change().dropna()

In [29]:
# Calculate historical metrics
ripple_mu = ripple_returns.mean()
ripple_sigma = ripple_returns.std()

dogecoin_mu = dogecoin_returns.mean()
dogecoin_sigma = dogecoin_returns.std()

In [30]:
# Simulation parameters
time_horizon = 365  # 1 year
steps = 1000  # Simulation steps
portfolio_weights = {'Ripple': 946.17, 'Dogecoin': 53.83}

In [31]:
# Simulate GBM for each cryptocurrency
def simulate_gbm(start_price, mu, sigma, time_horizon, steps):
    dt = time_horizon / steps
    price_path = [start_price]
    for _ in range(steps):
        price_path.append(price_path[-1] * np.exp((mu - 0.5 * sigma ** 2) * dt +
                                                  sigma * np.sqrt(dt) * np.random.normal()))
    return price_path

In [32]:
# Simulate prices
ripple_simulated = simulate_gbm(ripple_prices.iloc[-1], ripple_mu, ripple_sigma, time_horizon, steps)
dogecoin_simulated = simulate_gbm(dogecoin_prices.iloc[-1], dogecoin_mu, dogecoin_sigma, time_horizon, steps)

In [33]:
# Portfolio value at each time step
portfolio_value = [
    portfolio_weights['Ripple'] / ripple_prices.iloc[-1] * r +
    portfolio_weights['Dogecoin'] / dogecoin_prices.iloc[-1] * d
    for r, d in zip(ripple_simulated, dogecoin_simulated) 
]

In [34]:
# Create the plot
fig = go.Figure()

# Add the simulated prices line
fig.add_trace(go.Scatter(
    x=list(range(len(portfolio_value))),
    y=portfolio_value,
    mode='lines',
    name='Simulated Prices',
    line=dict(color='#206583', width=2)
))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': 'Portfolio Prediction using GBM',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [35]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Portfolio Prediction using GBM.html')

'Portfolio Prediction using GBM.html'

In [36]:
# Run simulation 10 times
simulations = []

simulations.append(portfolio_value)

for simulation in range(4):
    # Simulate prices
    ripple_simulated = simulate_gbm(ripple_prices.iloc[-1], ripple_mu, ripple_sigma, time_horizon, steps)
    dogecoin_simulated = simulate_gbm(dogecoin_prices.iloc[-1], dogecoin_mu, dogecoin_sigma, time_horizon, steps)

    # Portfolio value at each time step
    portfolio_value = [
        portfolio_weights['Ripple'] / ripple_prices.iloc[-1] * r +
        portfolio_weights['Dogecoin'] / dogecoin_prices.iloc[-1] * d
        for r, d in zip(ripple_simulated, dogecoin_simulated)
    ]

    simulations.append(portfolio_value)

In [39]:
# Create the plot
fig = go.Figure()

# Define a list of colors
colors = ['#0D3239', '#1A5465', '#206583', '#3487AB', '#3487AB']

# Add each simulated prices line with different colors
for i, prices in enumerate(simulations):
    fig.add_trace(go.Scatter(
        x=list(range(len(prices))),
        y=prices,
        mode='lines',
        name=f'Simulation {i+1}',
        line=dict(color=colors[i % len(colors)], width=2)
    ))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': 'Portfolio Multiple Prediction using GBM',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [38]:
plot(fig, auto_open=False, filename='Portfolio Multiple Prediction using GBM.html')

'Portfolio Multiple Prediction using GBM.html'